In [ ]:
import pandas as pd
import openai
import time
import re

In [ ]:
df = pd.read_excel("Antares Eazy Data .xlsx")
# df = pd.read_csv("real data lengkap.csv")
df

In [ ]:
# df = df.iloc[5001:9194].reset_index(drop=True)
# df

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key='[api_key]',
    api_version="2024-02-01",
    azure_endpoint = "https://openaifmseus.openai.azure.com/")

# deployment_name='gpt-35-turbo'
deployment_name='gpt-4o'

In [ ]:
def clean_output(pertanyaan, jawaban):
    # Hapus strip dari pertanyaan
    pertanyaan = pertanyaan.replace("-", "").strip()
    
    # Pisahkan jawaban menjadi paragraf-paragraf
    jawaban_paragraf = jawaban.split("\n")
    
    # Ambil hanya paragraf pertama
    jawaban_cleaned = []
    for line in jawaban_paragraf:
        if line.strip() == "":
            break
        jawaban_cleaned.append(line.strip())
    
    jawaban_cleaned = " ".join(jawaban_cleaned)
    
    return pertanyaan, jawaban_cleaned

def extract_customer_agent(text):
    prompt = f"""
    Anda adalah ahli bahasa yang mengevaluasi percakapan layanan pelanggan. 
    Analisis percakapan dan ekstrak pertanyaan dan jawaban dari percakapan berikut '{text}' yang:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak memerlukan data real-time, status pesanan, tiket, atau akses ke sistem internal
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV)
    
    Berikan hasil dalam format berikut:
    - **Pertanyaan**: [Tuliskan pertanyaan dari customer yang sudah dirapikan dalam bentuk kalimat tanya yang jelas]
    - **Jawaban**: [Tuliskan jawaban dari agen yang sudah dirapikan menjadi kalimat yang SINGKAT, padat, jelas, dan mudah dipahami tanpa menyebut nama customer]

    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas, berikan "No data".
    Jika terdapat nama orang pada jawaban, hapus nama dan ganti/sesuaikan dengan kalimatnya saja.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    """
    try:
        chat_log = [{'role': 'user', 'content': prompt}]
        response = client.chat.completions.create(model=deployment_name, messages=chat_log)
        output = response.choices[0].message.content.strip()
        print(f"Output: {output}")

        pertanyaan, jawaban = "No data", "No data"
        if "**Pertanyaan**:" in output and "**Jawaban**:" in output:
            parts = output.split("**Jawaban**:")
            pertanyaan = parts[0].replace("**Pertanyaan**:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        
        # Cleaning the output
        pertanyaan, jawaban = clean_output(pertanyaan, jawaban)

        return pertanyaan, jawaban
    except Exception as e:
        print(f"Error: {e}")
        return "Error", "Error"

def analyze_conversations(df, batch_size=500, sleep_time=60):
    total_rows = len(df)
    
    # Menambahkan kolom baru untuk hasil ekstraksi
    if 'Pertanyaan' not in df.columns:
        df['Pertanyaan'] = ""
    if 'Jawaban' not in df.columns:
        df['Jawaban'] = ""

    for i in range(total_rows):
        try:
            text = df.iloc[i]['Conversation']
            
            if pd.isna(text) or text.strip() == "":
                df.loc[i, 'Pertanyaan'] = "No data"
                df.loc[i, 'Jawaban'] = "No data"
                continue
            
            # Ekstraksi pertanyaan customer dan jawaban agen
            pertanyaan, jawaban = extract_customer_agent(text)
            df.loc[i, 'Pertanyaan'] = pertanyaan
            df.loc[i, 'Jawaban'] = jawaban
            
        except Exception as e:
            print(f"[{i}]> Error: {e}")
            df.loc[i, 'Pertanyaan'] = "Error"
            df.loc[i, 'Jawaban'] = "Error"
        
        # Batch processing untuk menghindari batasan API
        if (i + 1) % batch_size == 0:
            print(f"Processed {i + 1}/{total_rows} rows. Sleeping for {sleep_time} seconds...")
            time.sleep(sleep_time)
    
    return df


In [ ]:
%%time
analyze_conversations(df)
df

In [ ]:
df.to_csv("test_qna_myindibiz.csv", index=False) 

In [ ]:
def clean_output(pertanyaan, jawaban):
    pertanyaan = pertanyaan.replace("-", "").strip()
    jawaban_paragraf = jawaban.split("\n")
    jawaban_cleaned = []
    for line in jawaban_paragraf:
        if line.strip() == "":
            break
        jawaban_cleaned.append(line.strip())
    jawaban_cleaned = " ".join(jawaban_cleaned)
    return pertanyaan, jawaban_cleaned

def extract_customer_agent(text):
    prompt = f"""
    Anda adalah ahli bahasa yang mengevaluasi percakapan layanan pelanggan. 
    Analisis percakapan dan ekstrak pertanyaan dan jawaban dari percakapan berikut '{text}' yang:
    1. Pertanyaan berkaitan dengan produk, fitur, kebijakan, informasi layanan, atau sistem secara umum
    2. Pertanyaan bisa dijawab oleh AI/chatbot tanpa perlu informasi spesifik pelanggan
    3. Pertanyaan tidak memerlukan data real-time, status pesanan, tiket, atau akses ke sistem internal
    4. Jawaban yang diberikan agent sesuai untuk pertanyaan tersebut
    5. Pertanyaan berkaitan dengan cara melakukan sesuatu atau prosedur
    7. Pertanyaan tidak memerlukan jawaban/tindakan manual oleh agen/manusia
    8. Jawaban tidak membutuhkan data pribadi customer
    9. Jawaban tidak meminta customer mununggu untuk info/proses lebih lanjut
    10. Pertanyaan dan jawabannya tidak mengandung perihal nomor tiket (DOA) atau invoice (INV)
    
    Berikan hasil dalam format berikut:
    - **Pertanyaan**: [Tuliskan pertanyaan dari customer yang sudah dirapikan dalam bentuk kalimat tanya yang jelas]
    - **Jawaban**: [Tuliskan jawaban dari agen yang sudah dirapikan menjadi kalimat yang SINGKAT, padat, jelas, dan mudah dipahami tanpa menyebut nama customer]

    Jika percakapan tidak berisi pertanyaan atau jawaban yang jelas, berikan "No data".
    Jika terdapat nama orang pada jawaban, hapus nama dan ganti/sesuaikan dengan kalimatnya saja.
    Jika pertanyaan adalah tata cara berikan secara step by step tetapi tetap singkat dan jelas.

    """
    try:
        chat_log = [{'role': 'user', 'content': prompt}]
        response = client.chat.completions.create(model=deployment_name, messages=chat_log)
        output = response.choices[0].message.content.strip()

        pertanyaan, jawaban = "No data", "No data"
        if "**Pertanyaan**:" in output and "**Jawaban**:" in output:
            parts = output.split("**Jawaban**:")
            pertanyaan = parts[0].replace("**Pertanyaan**:", "").strip()
            jawaban = parts[1].strip() if len(parts) > 1 else "No data"
        
        pertanyaan, jawaban = clean_output(pertanyaan, jawaban)

        return pertanyaan, jawaban
    except Exception as e:
        print(f"Error: {e}")
        return "Error", "Error"

def reprocess_errors(df):
    for i in df.index:
        if df.loc[i, 'Pertanyaan'] == "Error" or df.loc[i, 'Jawaban'] == "Error":
            try:
                text = df.loc[i, 'Conversation']
                if pd.isna(text) or text.strip() == "":
                    df.loc[i, 'Pertanyaan'] = "No data"
                    df.loc[i, 'Jawaban'] = "No data"
                    continue
                
                pertanyaan, jawaban = extract_customer_agent(text)
                df.loc[i, 'Pertanyaan'] = pertanyaan
                df.loc[i, 'Jawaban'] = jawaban

            except Exception as e:
                print(f"Reprocessing Error at row {i}: {e}")
                df.loc[i, 'Pertanyaan'] = "Error"
                df.loc[i, 'Jawaban'] = "Error"
    
    return df

In [ ]:
%%time
df = reprocess_errors(df)
df

In [ ]:
# Filter the DataFrame for rows where 'Pertanyaan' or 'Jawaban' is "Error"
error_rows = df[(df['Pertanyaan'] == "Error") | (df['Jawaban'] == "Error")]
    
    # Display the filtered DataFrame
error_rows

In [ ]:
df.to_csv('updated_gpt_qnatigabulanpls.csv', index=False)

In [ ]:
def clean_text_pertanyaan(text):
    # Pisahkan teks menjadi paragraf berdasarkan newline ganda dan ambil paragraf terakhir
    paragraphs = text.strip().split('\n\n')
    return paragraphs[-1].strip()

# Terapkan fungsi untuk mendapatkan paragraf terakhir ke kolom 'Pertanyaan'
df['cleaned_pertanyaan'] = df['Pertanyaan'].apply(clean_text_pertanyaan)

In [ ]:
df

In [ ]:
df.to_csv('updated_qna_mytensupdate.csv', index=False)

In [ ]:
# Menghapus baris yang berisi "No Data"
df = df[(df['cleaned_pertanyaan'] != 'No data') & (df['Jawaban'] != 'No data')]

# Reset indeks
df = df.reset_index(drop=True)

# Menampilkan dataframe yang telah dibersihkan
df

In [ ]:
df.to_csv('antareseazy_baru.csv', index=False)